In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot

In [3]:
#new_model = tf.keras.models.load_model("D:/Modelos/EfficientNet_B1_IMAGENET_IC")

In [4]:
#new_model.summary()

In [5]:
#carrega os arrays de imagens e de classes
X = np.load("D:/Displasia/displasiaDataAug/displasiaOriginalDataX_teste3_GLCM.npy")
Y = np.load("D:/Displasia/displasiaDataAug/displasiaOriginalDataYOneHot_teste3.npy")
Z = np.load("D:/Displasia/displasiaDataAug/displasiaOriginalDataX_teste3_SCM.npy")
#dataX = np.load("Image Arrays/displasiaGeneratedDataX.npy")
#dataY = np.load("Image Arrays/displasiaGeneratedDataY.npy")

X = np.append(X,Z,axis=1)

#shape dos arrays
X.shape, Y.shape

((75776, 32), (75776, 4))

In [6]:
from sklearn.model_selection import train_test_split

#divide os dados em 80% para treino e 20% para teste
train_X, test_X, train_label, test_label = train_test_split(X, Y, test_size=0.2, random_state=13)

#divide os dados de treino em 80% para treino e 20% para validação
#train_X, valid_X, train_label, valid_label = train_test_split(train_X, train_label, test_size=0.2, random_state=13)

#mostra os shapes resultantes das divisões
print("Training:   ", train_X.shape, train_label.shape)
#print("Validation: ", valid_X.shape, valid_label.shape)
print("Testing:    ", test_X.shape, test_label.shape)

Training:    (60620, 32) (60620, 4)
Testing:     (15156, 32) (15156, 4)


In [7]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
#kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
kfold = KFold(n_splits=5, shuffle=True, random_state=7)
cvscores = []
num_kfold=1
#from tensorflow.keras.layers import Activation, Dense
#new_model = GlobalAveragePooling2D()(model)
#drop = Dropout(0.2)(new_model)
#new_model2 = Dense(4,activation='relu')(drop)

#new_model3 = Model(inputs=model.input, outputs=new_model)
#layers.GlobalAveragePooling2D(
#    data_format=None, keepdims=False, #**kwargs
#)

mean = [[[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]]]

std = [[[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]]]

for train, test in kfold.split (X, Y):
    
    clf=RandomForestClassifier(n_estimators=100)
    #train_labels2=train_labels[:49984]
    clf.fit(X[train],Y[train])
    
    y_pred=clf.predict(X[test])
    print("Accuracy:",metrics.accuracy_score(Y[test], y_pred))
    #faz a predição das imagens

    from keras.utils import to_categorical

    predicted_classes = y_pred

    predicted_classes = np.argmax(np.round(predicted_classes),axis=1)

    predicted_classes = to_categorical(predicted_classes)

#    predicted_classes.shape, test_label.shape

    num_classes = 4

    target_names = ["Class {}".format(i) for i in range(num_classes)]
    print(classification_report(Y[test], predicted_classes, target_names=target_names))

    #converte os arrays de classe de one-hot para númerico
    test_label = np.argmax(Y[test], axis=1)
    predicted_classes = np.argmax(predicted_classes, axis=1)

    #gera a matriz de confusão a partir da predição feita
    cm = confusion_matrix(test_label, predicted_classes)

    #gera os valores de falso positivo, falso negativo, verdadeiro positivo e verdadeiro negativo
    fp = cm.sum(axis=0) - np.diag(cm)  
    fn = cm.sum(axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tn = cm.sum() - (fp + fn + tp)

    #gera a sensitividade, especificidade, acurácia e precisão
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy    = (tp + tn) / (tp + tn + fp + fn)
    precision   = tp / (tp + fp)
    f1_score = 2/((1/precision)+(1/sensitivity))

    #mostra os resultados
    print("Precision:  ", precision)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)
    print("F1-score:   ", f1_score)
    print("Accuracy:   ", accuracy)
    
    x = []
    x.append(precision)
    x.append(sensitivity)
    x.append(specificity)
    x.append(f1_score)
    x.append(accuracy)
    
    x = np.asarray(x)
    
    x = x.transpose()
    print()
    print(x)
    
    for i in range(4):
        for j in range(5):
            mean[i][j].append(x[i][j])
            std[i][j].append(x[i][j])

Accuracy: 0.8775402480865664
              precision    recall  f1-score   support

     Class 0       0.74      0.99      0.85      3867
     Class 1       0.99      0.88      0.93      3755
     Class 2       0.99      0.88      0.93      3686
     Class 3       0.99      0.87      0.93      3848

   micro avg       0.91      0.91      0.91     15156
   macro avg       0.93      0.91      0.91     15156
weighted avg       0.93      0.91      0.91     15156
 samples avg       0.91      0.91      0.91     15156

Precision:   [0.74378882 0.99427711 0.99056891 0.98793053]
Sensitivity: [0.99094906 0.87909454 0.88334238 0.87214137]
Specificity: [0.88307202 0.99833348 0.9972973  0.99637425]
F1-score:    [0.84976161 0.93314488 0.93388785 0.92643202]
Accuracy:    [0.91059646 0.96879124 0.969583   0.96483241]

[[0.74378882 0.99094906 0.88307202 0.84976161 0.91059646]
 [0.99427711 0.87909454 0.99833348 0.93314488 0.96879124]
 [0.99056891 0.88334238 0.9972973  0.93388785 0.969583  ]
 [0.98793053

In [8]:
mean = np.asarray(mean)
std = np.asarray(std)
for i in mean:
    print(np.mean(i,axis=1))
print()
for i in std:
    print(np.std(i,axis=1))

[0.73891919 0.99227584 0.88316005 0.84704567 0.91043338]
[0.99450595 0.8774094  0.99838052 0.9322881  0.96814294]
[0.99267733 0.87421099 0.99785475 0.92966933 0.96691561]
[0.98618075 0.88215154 0.99588664 0.93124655 0.96743036]

[0.00471399 0.00181236 0.00168691 0.00265658 0.00126775]
[0.00235496 0.00464586 0.00070003 0.00292803 0.00126836]
[0.00173721 0.00660779 0.00049373 0.00353964 0.00206051]
[0.00416824 0.00592561 0.00122139 0.00249621 0.00144879]


In [9]:
#import keras
#intermediate_layer_model = keras.Model(inputs=new_model.input,
#                                       outputs=new_model.get_layer('flatten').output)
#intermediate_output = intermediate_layer_model.predict(valid_X)

In [10]:
#newArr = np.append(intermediate_output, GLCM_features2 , axis=1)

In [11]:
#from sklearn.ensemble import RandomForestClassifier

In [12]:
#clf=RandomForestClassifier(n_estimators=100)
#train_labels2=train_labels[:49984]
#clf.fit(train_X,train_label)

In [13]:
#intermediate_layer_model_2 = keras.Model(inputs=new_model.input,
#                                       outputs=new_model.get_layer('flatten').output)
#intermediate_output_2 = intermediate_layer_model_2.predict(test_X)

In [14]:
#newArr_test = np.append(intermediate_output_2, GLCM_features2_test , axis=1)

In [15]:
#y_pred=clf.predict(test_X)

In [16]:
#from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
#print("Accuracy:",metrics.accuracy_score(test_label, y_pred))

In [17]:
#faz a predição das imagens

#from keras.utils import to_categorical

#predicted_classes = y_pred

#predicted_classes = np.argmax(np.round(predicted_classes),axis=1)

#predicted_classes = to_categorical(predicted_classes)

#predicted_classes.shape, test_label.shape

In [18]:
#mostra as estatísticas de cada classe
#num_classes = 4

#from sklearn.metrics import classification_report
#target_names = ["Class {}".format(i) for i in range(num_classes)]
#print(classification_report(test_label, predicted_classes, target_names=target_names))

In [19]:
#from sklearn.metrics import confusion_matrix

#converte os arrays de classe de one-hot para númerico
#test_label = np.argmax(test_label, axis=1)
#predicted_classes = np.argmax(predicted_classes, axis=1)

#gera a matriz de confusão a partir da predição feita
#cm = confusion_matrix(test_label, predicted_classes)

#gera os valores de falso positivo, falso negativo, verdadeiro positivo e verdadeiro negativo
#fp = cm.sum(axis=0) - np.diag(cm)  
#fn = cm.sum(axis=1) - np.diag(cm)
#tp = np.diag(cm)
#tn = cm.sum() - (fp + fn + tp)

#gera a sensitividade, especificidade, acurácia e precisão
#sensitivity = tp / (tp + fn)
#specificity = tn / (tn + fp)
#accuracy    = (tp + tn) / (tp + tn + fp + fn)
#precision   = tp / (tp + fp)
#f1_score = 2/((1/precision)+(1/sensitivity))

#mostra os resultados
#print("Precision:  ", precision)
#print("Sensitivity:", sensitivity)
#print("Specificity:", specificity)
#print("F1-score:   ", f1_score)
#print("Accuracy:   ", accuracy)